In [4]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelBinarizer

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization

In [5]:
def get_data():
    """Returns the data as a pandas dataframe"""
    try:
        df = pd.read_csv('training_data/sign_mnist_train.csv')
    except:
        df = pd.read_csv(r'HandsignInterpreter\training_data\sign_mnist_train.csv')
    return df

def get_test_data():
    """Returns the data as a pandas dataframe"""
    try:
        df = pd.read_csv('training_data/sign_mnist_test.csv')
    except:
        df = pd.read_csv(r'HandsignInterpreter\training_data\sign_mnist_test.csv')
    return df

In [6]:
def cnn_train():
    df = get_data()
    df_test = get_test_data()
    classes_train = df['label']
    classes_test = df_test['label']
    del df['label']
    del df_test['label']

    label_binarizer = LabelBinarizer()
    classes_train = label_binarizer.fit_transform(classes_train)
    classes_test = label_binarizer.fit_transform(classes_test)

    vector_train = df.values
    vector_test = df_test.values
    vector_train = vector_train / 255
    vector_test = vector_test / 255
    vector_train = vector_train.reshape(-1, 28, 28, 1)
    vector_test = vector_test.reshape(-1, 28, 28, 1)

    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    datagen.fit(vector_train)

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

    model = Sequential()
    model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(Flatten())
    model.add(Dense(units = 512 , activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units = 24 , activation = 'softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    model.summary()

    history = model.fit(datagen.flow(vector_train, classes_train, batch_size = 128), epochs = 20, validation_data = (vector_test, classes_test), callbacks = [learning_rate_reduction])
    print("Accuracy of the model is - " , model.evaluate(vector_test, classes_test)[1]*100 , "%")

In [7]:
cnn_train()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 75)       300       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 75)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 50)       2

2022-12-13 16:54:20.059786: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
215/215 [==============================] - 29s 133ms/step - loss: 1.0417 - accuracy: 0.6746 - val_loss: 3.6133 - val_accuracy: 0.1141 - lr: 0.0010
Epoch 2/20
215/215 [==============================] - 30s 141ms/step - loss: 0.2021 - accuracy: 0.9352 - val_loss: 0.9174 - val_accuracy: 0.7235 - lr: 0.0010
Epoch 3/20
215/215 [==============================] - 31s 145ms/step - loss: 0.0902 - accuracy: 0.9721 - val_loss: 0.1569 - val_accuracy: 0.9509 - lr: 0.0010
Epoch 4/20
215/215 [==============================] - 32s 150ms/step - loss: 0.0634 - accuracy: 0.9803 - val_loss: 0.1929 - val_accuracy: 0.9364 - lr: 0.0010
Epoch 5/20
215/215 [==============================] - 33s 153ms/step - loss: 0.0406 - accuracy: 0.9862 - val_loss: 0.0253 - val_accuracy: 0.9919 - lr: 0.0010
Epoch 6/20
215/215 [==============================] - 32s 149ms/step - loss: 0.0334 - accuracy: 0.9887 - val_loss: 0.0221 - val_accuracy: 0.9941 - lr: 0.0010
Epoch 7/20
215/215 [==============================] 